# [手把手 | 基于TextRank算法的文本摘要（附Python代码）](https://mp.weixin.qq.com/s?__biz=MjM5MTQzNzU2NA==&mid=2651666276&idx=3&sn=fbd030dda4318ca328fcf918fc1df1e3&chksm=bd4c10f78a3b99e1b6ebfe362a969f36f39e72ce93336bf3fe4fa03b237120156a82c8becfd4&mpshare=1&scene=1&srcid=0926aQxlE7uoNlUkHMjmqwpV&sharer_sharetime=1569463898414&sharer_shareid=7429dfbe8eeefb16193f3b889173524e&key=1d1ca7b6df234b6eb8721eb961bac936b31eea52d6df0f9da7f23ed3ac6a644973c0ce22b6d73db01420f94551b51ba4ff3373167795ed451f3c89779f5c0ab3936d1403bee433d2c0815daaab647ab5&ascene=1&uin=MTE2NTM1NQ%3D%3D&devicetype=Windows+10&version=62070141&lang=zh_CN&pass_ticket=FjR0fPMrSxrRHa7VbnHhBJ%2BAFLgOTXEKsBcxoKN7PN8%3D)
## 问题背景介绍
作为一个网球爱好者，我一直试图通过对尽可能多的网球新闻的阅读浏览来使自己随时了解这项运动的最新情况。然而，事实证明这已经是一项相当困难的工作！花费太多的资源和时间是一种浪费。
因此，我决定设计一个系统，通过扫描多篇文章为我提供一个要点整合的摘要。如何着手做这件事？这就是我将在本教程中向大家展示的内容。我们将在一个爬取得到的文章集合的文本数据集上应用TextRank算法，以创建一个漂亮而简洁的文章摘要。



1. 导入所需的库
首先导入解决本问题需要的库


In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\壹心理\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

2. 读入数据

现在读取数据，在上文我已经提供了数据集的下载链接。


In [5]:
df = pd.read_csv("tennis_articles_v4.csv")


3. 检查数据

让我们快速了解以下数据。



In [7]:
df.head()


,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


数据集有三列，分别是‘article_id’，‘article_text’，和‘source’。我们对‘article_text’列的内容最感兴趣，因为它包含了文章的文本内容。让我们打印一些这个列里的变量的值，具体看看它们是什么样。


In [8]:
df['article_text'][0]


"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

现在我们有两种选择，一个是总结单个文章，一个是对所有文章进行内容摘要。为了实现我们的目的，我们继续后者。

4. 把文本分割成句子

下一步就是把文章的文本内容分割成单个的句子。我们将使用nltk库中的sent_tokenize( )函数来实现。


In [16]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list



打印出句子列表中的几个元素。


In [17]:
sentences[:5]


['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl."]

5. 下载GloVe词向量



GloVe词向量是单词的向量表示。这些词向量将用于生成表示句子的特征向量。我们也可以使用Bag-of-Words或TF-IDF方法来为句子生成特征，但这些方法忽略了单词的顺序，并且通常这些特征的数量非常大。



我们将使用预训练好的Wikipedia 2014 + Gigaword 5 （补充链接）GloVe向量，文件大小是822 MB。



GloVe词向量下载链接：

https://nlp.stanford.edu/data/glove.6B.zip


In [20]:
# extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [16]:
len(word_embeddings)


现在我们在字典中存储了400000个不同术语的词向量。



6. 文本预处理



尽可能减少文本数据的噪声是一个好习惯，所以我们做一些基本的文本清洗（包括移除标点符号、数字、特殊字符，统一成小写字母）。


In [20]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z"," ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


去掉句子中出现的停用词（一种语言的常用词——is，am，of，in等）。如果尚未下载nltk-stop，则执行以下代码行：


In [20]:
nltk.download('stopwords')


In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


接下来定义移除我们的数据集中停用词的函数。


In [20]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = "".join([i for i in sen if i not in stop_words])
    return sen_new



In [20]:
#remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split() for r in clean_sentences)]


我们将在GloVe词向量的帮助下用clean_sentences（程序中用来保存句子的列表变量）来为我们的数据集生成特征向量。



7. 句子的特征向量



现在，来为我们的句子生成特征向量。我们首先获取每个句子的所有组成词的向量（从GloVe词向量文件中获取，每个向量大小为100个元素），然后取这些向量的平均值，得出这个句子的合并向量为这个句子的特征向量。


In [22]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

SyntaxError: invalid syntax (<ipython-input-22-2623386baeca>, line 5)


8. 相似矩阵准备



下一步是找出句子之间的相似性，我们将使用余弦相似性来解决这个问题。让我们为这个任务创建一个空的相似度矩阵，并用句子的余弦相似度填充它。



首先定义一个n乘n的零矩阵，然后用句子间的余弦相似度填充矩阵，这里n是句子的总数。

In [20]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])


将用余弦相似度计算两个句子之间的相似度。


In [20]:
from sklearn.metrics.pairwise import cosine_similarity


用余弦相似度初始化这个相似度矩阵。


In [23]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


NameError: name 'cosine_similarity' is not defined

9. 应用PageRank算法 



在进行下一步之前，我们先将相似性矩阵sim_mat转换为图结构。这个图的节点为句子，边用句子之间的相似性分数表示。在这个图上，我们将应用PageRank算法来得到句子排名。

In [23]:
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)


10. 摘要提取



最后，根据排名提取前N个句子，就可以用于生成摘要了。


In [23]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [23]:
# Extract top 10 sentences as the summary
for i in range(10):
    print(ranked_sentences[i][j])


现在我们实现了一个棒极了、整齐的、简洁、有用的文章总结！



五、下一步是什么？



自动文本摘要是一个热门的研究课题，在本文中我们仅仅讨论了冰山一角。展望未来，我们将探索抽象文本摘要技术，其中深度学习扮演着重要的角色。此外，我们还可以研究下面的文本摘要任务：



1. 问题导向：

多领域文本摘要

单个文档的摘要

跨语言文本摘要

（文本来源是一种语言，文本总结用另一种语言）



2. 算法导向：

应用RNN和LSTM的文本摘要

应用加强学习的文本摘要

应用生成对抗神经网络（GAN）的文本摘要


后记



我希望这篇文章能帮助你理解自动文本摘要的概念。它有各种各样的应用案例，并且已经产生了非常成功的应用程序。无论是在您的业务中利用，还是仅仅为了您自己的知识，文本摘要是所有NLP爱好者都应该熟悉的方法。              



我将在以后的文章中尝试使用高级技术介绍抽象文本摘要技术。同时，请随时使用下面的评论部分让我知道你对这篇文章的想法或任何问题。



数据集下载链接：

https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/tennis_articles_v4.csv



算法代码链接：

https://github.com/prateekjoshi565/textrank_text_summarization



相关报道：

https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/

